In [1]:
%matplotlib inline
import pandas as pd
import xgboost as xgb

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

iris_data = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_target = pd.Series(iris.target)

In [3]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(iris_data, iris_target, test_size=0.2, shuffle=True)

In [4]:
params = {'eta': [0.01, 0.1, 1.0], 'gamma': [0], 
                  'n_estimators': [100], 'max_depth':[2], 'min_child_weigh': [1], 
                  'nthread': [2] }

In [5]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.917, total=   1.2s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.958, total=   0.3s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s


[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.750, total=   0.4s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.3s
[CV] eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.01, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=1.000, total=   0.2s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.875, total=   0.3s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.958, total=   0.2s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, 

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    5.9s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
       

In [6]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % (row['mean'], row['std']*2, row['params']))

score: 0.925 +/-0.1856, params: {'eta': 0.01, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 1.0, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}


In [7]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.9250
{'eta': 0.01, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}


In [8]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9000


In [9]:
params = {'eta': [0.1], 'gamma': [0], 
                  'n_estimators': [50, 100, 200], 'max_depth':[2], 'min_child_weigh': [1], 
                  'nthread': [2] }

In [10]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=0.875, total=   0.2s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=0.958, total=   0.4s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=0.750, total=   0.1s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.2s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=50, nthread=2, score=1.000, total=   0.1s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.875, total=   0.3s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.958, total=   1.5s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2, score=0.750, total=   0.7s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=100, nthread=2 
[CV]  eta=0.1, gamma=0,

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   11.1s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
       

In [11]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % (row['mean'], row['std']*2, row['params']))

score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 100, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}


In [12]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.9167
{'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 50, 'nthread': 2}


In [13]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9667


In [14]:
params = {'eta': [0.1], 'gamma': [0, 0.1, 0.2], 
                  'n_estimators': [200], 'max_depth':[2, 4], 'min_child_weigh': [1, 2], 
                  'nthread': [2] }

In [15]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=0.875, total=   0.8s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=0.958, total=   0.4s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s


[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=0.750, total=   0.6s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=1.000, total=   2.4s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=1.000, total=   1.0s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2, score=0.875, total=   1.9s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2, score=0.958, total=   2.2s
[CV] eta=0.1, gamma=0, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma

[CV]  eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=1, n_estimators=200, nthread=2, score=1.000, total=   1.6s
[CV] eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2, score=0.875, total=   1.6s
[CV] eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2, score=0.958, total=   2.0s
[CV] eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2, score=0.750, total=   0.4s
[CV] eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 
[CV]  eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2, score=1.000, total=   0.9s
[CV] eta=0.1, gamma=0.2, max_depth=2, min_child_weigh=2, n_estimators=200, nthread=2 


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     verbosity=None),
       

In [16]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % (row['mean'], row['std']*2, row['params']))

score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 2, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 4, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0, 'max_depth': 4, 'min_child_weigh': 2, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 2, 'min_child_weigh': 2, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 4, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}
score: 0.917 +/-0.1900, params: {'eta': 0.1, 'gamma': 0.1, 'max_depth': 4, 'min_chil

In [17]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.9167
{'eta': 0.1, 'gamma': 0, 'max_depth': 2, 'min_child_weigh': 1, 'n_estimators': 200, 'nthread': 2}


In [18]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9000
